In [1]:
import xnat
import zipfile
#import mysql.connector
#import json
import os
import pydicom as dicom

In [2]:
# set up XNAT login credentials here
# .......................
xnatUrl = 'http://localhost:80'  #change me! e.g. 'http://localhost:8081/'
#xnatUrl = 'https://xnat.bmia.nl'
xnatUser = 'admin'  #change me!
#xnatUser = 'leonardwee'  #change me!
xnatPass = 'admin'  #change me!
xnatProject = 'argostest02'  #change me!

In [3]:
# ----------------------------------------------------------------------------------- create a zip function
#fileList : a python list containing the path to each file to put packed into the zip archive
def createZip(fileList, fileName, omitPrefixFolder=None):
    fileZip = zipfile.ZipFile(fileName, 'w', zipfile.ZIP_DEFLATED)

    for myFile in fileList:
        archFileName = myFile
        if omitPrefixFolder is not None:
            archFileName = myFile.replace(omitPrefixFolder, '')
        try:
            fileZip.write(myFile, archFileName)
        except:
            print("Could not find file: " + myFile)

    fileZip.close()

    return fileName

In [4]:
# ----------------------------------------------------------------------------------- create an xnat upload function
#conn : is the connector handle to an XNAT service
#zipFilePath : is the fullPath to the zipped dicom data object
#project : is the unique identifier of the xnat project - see your setting for xnatProject above
#subject : is the unique identifier of the patient
#studyDescription : is the identifier of the study eg "CT"
def uploadData(conn, zipFilePath, project, subject, studyDescription):
    conn.services.import_(zipFilePath, content_type='application/zip',
                          project=project,
                          subject=subject,
                          experiment=studyDescription, overwrite="delete")

In [5]:
# Connecting to XNAT service
connection = xnat.connect(xnatUrl, user=xnatUser, password=xnatPass)

[WARNING] Found an unsupported version (1.8.2), trying 1.7 compatible model builder


In [6]:
#script starts here
dataLocation = 'C:\\argos\\ctp_processing\\ctp_roi_renamed' #change me!

patientList = [ f.path for f in os.scandir(dataLocation) if f.is_dir() ]

In [7]:
for ff in patientList:
    patid = ff.split('\\')[-1]
    targetFile = dataLocation+'\\'+patid+'.zip' ##change me!
    listOfFiles = [os.path.join(dp, f) for dp, dn, filenames in os.walk(ff) for f in filenames]
    #this = dicom.read_file(listOfFiles[0], force=True)
    xnatSubject = patid
    xnatStudyDescription = 'CT' #or change to this.StudyDescription
    zipFileName = createZip(listOfFiles,targetFile)
    print(zipFileName)
    try:
        #uploadData(connection, zipFileName,xnatProject,xnatSubject,xnatStudyDescription)
        prearchive_session = connection.services.import_(zipFileName, project=xnatProject, destination='/prearchive')
        os.remove(zipFileName)
    except:
        print("Could not upload study : "+xnatSubject)

# Close XNAT connection
connection.disconnect()

print('FINISHED!')

[WARNING] Found unexpect content type, but assuming zip based on the extension


C:\argos\ctp_processing\ctp_roi_renamed\LUNG1-003.zip


[WARNING] Found unexpect content type, but assuming zip based on the extension


C:\argos\ctp_processing\ctp_roi_renamed\LUNG1-004.zip


[WARNING] Found unexpect content type, but assuming zip based on the extension


C:\argos\ctp_processing\ctp_roi_renamed\LUNG1-005.zip
FINISHED!


In [8]:
# Close XNAT connection
connection.disconnect()